# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [196]:
import numpy as np


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [197]:
import os

def load_data(folder_path):
    texts = []
    # Walk through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):  # Only process .txt files
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
                texts.append(text)
    return texts


In [198]:
stopwords = stopwords.words("english")
print(stopwords)
stopwords.extend(['et', 'al','figure', 'paper', 'type', 'table','id','citation','subject','output','fig','matrix','variable','ipe'])
print(stopwords)

stopwords.extend(['student', 'teacher', 'educator', 'class', 'course', 'ai', 'artificial', 'intelligence'])
stopwords.extend([
    'research', 'study', 'data', 'result', 'method', 'analysis', 'model', 'system', 'learning', 'education','category','cluster', 'respondent',
    'use', 'using', 'used', 'based', 'implementation', 'gai', 'gen', 'genai', 'gpt', 'chatgpt','writing','faculty','survey','llm','llms','participant','image','perception', 'effect', 'factor', 'university', 'group', 'case',
    'response', 'influence'
])
print(stopwords)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [199]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [200]:
#data = load_data("data/ushmm_dn.json")["texts"]

#print (data[0][0:90])

# Use your folder path
folder_path = "data"
data = load_data(folder_path)

# Print first 90 characters of first text to verify
if data:
    print(data[0][0:90])

A Structural equation modeling analysis of generative AI chatbots adoption among students 


In [202]:
print(data[15][0:150])


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):  
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[15][0:150])


Evaluating generative AI integration in Saudi Arabian education: a mixed-methods study

Abstract
 
Incorporating generative artificial intelligence (G
evaluate generative integration saudi arabian education mixed method study incorporate generative artificial intelligence education become crucial con


In [203]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['structural', 'equation', 'model', 'analysis', 'generative', 'chatbot', 'adoption', 'student', 'educator', 'high', 'education', 'era', 'artificial', 'intelligence', 'reshape', 'educational', 'paradigm', 'study', 'explore', 'base']


In [204]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]


data_words_nostops = remove_stopwords(data_words)

#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words_nostops, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words_nostops], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words_nostops)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['structural_equation', 'generative', 'chatbot', 'adoption', 'high', 'era', 'reshape', 'educational', 'paradigm', 'explore', 'base', 'chatbot', 'adoption', 'high', 'employ', 'approach', 'focus', 'develop', 'validate', 'comprehensive']


In [205]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [80]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [206]:
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
#                                            id2word=id2word,
#                                            num_topics=4,
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha="auto")


lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus[:-1],
    id2word=id2word,
    num_topics=4,
    passes=20,
    alpha='asymmetric',
    eta=0.2
)


In [207]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.47337997), (1, 0.31922764), (2, 0.14275381), (3, 0.064638615)]
[(0, 0.47337997), (1, 0.31922764), (2, 0.14275381), (3, 0.064638615)]


In [208]:
lda_model.save("models/test_model.model")

In [209]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [210]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.47337902), (1, 0.31922787), (2, 0.14275327), (3, 0.06463984)]
[(0, 0.47337902), (1, 0.31922787), (2, 0.14275327), (3, 0.06463984)]


## Vizualizing the Data

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

In [16]:
print(data[64][0:90])

Challenges and Opportunities of Generative AI for Higher Education as Explained by ChatGPT
